In [ ]:
from utils import * 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import itertools
from pprint import pprint
import joblib
import statistics

from xgboost import XGBClassifier, XGBRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import KFold, cross_validate, GridSearchCV, cross_val_score, RandomizedSearchCV 
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error , mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, f1_score, matthews_corrcoef
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin
from sklearn.base import clone
from sklearn.model_selection._split import check_cv

## Загружаем данные 

In [2]:
train_labels = pd.read_csv('../data/processed/train_labels.csv', index_col = 'CASRN')
test_labels = pd.read_csv('../data/processed/test_labels.csv', index_col = 'CASRN')
train_labels.shape, test_labels.shape

((8221, 6), (2849, 6))

In [3]:
train_Hfeatures = pd.read_csv('../data/Hmodel_features_combined/train_Hfeatures.csv', index_col = 'CASRN')
train_Hfeatures.shape

(8221, 100)

In [4]:
train_Hfeatures.head(1)

,EPA_RF_ecfp6bits-1,EPA_RF_ecfp6bits-2,EPA_RF_ecfp6bits-3,EPA_RF_ecfp6counts-1,EPA_RF_ecfp6counts-2,EPA_RF_ecfp6counts-3,EPA_RF_maccs-1,EPA_RF_maccs-2,EPA_RF_maccs-3,EPA_RF_mordred-1,...,Toxic_svm_ecfp6bits-1,Toxic_svm_ecfp6counts-1,Toxic_svm_maccs-1,Toxic_svm_mordred-1,Toxic_svm_rdkit2d-1,Toxic_xgboost_ecfp6bits-1,Toxic_xgboost_ecfp6counts-1,Toxic_xgboost_maccs-1,Toxic_xgboost_mordred-1,Toxic_xgboost_rdkit2d-1
CASRN,,,,,,,,,,,,,,,,,,,,,
23233-88-7,0.11421,0.23115,0.516881,0.106932,0.231361,0.535563,0.084305,0.184793,0.545046,0.090417,...,0.595123,0.693394,0.271051,0.490151,0.46636,0.660376,0.721718,0.298831,0.484635,0.289508


Моделирование эндпоинтов 
В этом разделе мы будем моделировать три конечные точки с использованием четырёх различных алгоритмов: KNN (k-Nearest Neighbors), XGBoost, SVM и Random Forest.

Будем использовать данные характеристик молекул и их меток для построения моделей.

In [ ]:
knn_reg = KNeighborsRegressor()
svr = SVR()
xgb_reg = XGBRegressor(random_state =123, n_jobs=6 ,objective ='reg:squarederror')
rf_reg =  RandomForestRegressor(random_state =123, n_jobs=6)

svc = SVC(random_state =42)
knn_clf = KNeighborsClassifier()
xgb_clf = XGBClassifier(random_state =123, n_jobs=6)
rf_clf =  RandomForestClassifier(random_state =123, n_jobs=6)

### Определение пространства поиска гиперпараметров

In [6]:
knn_grid_parameters = {'n_neighbors': [5,9,15,19,25,35,45,55,71,101,151], 'weights': ['distance'],'p': [1,2]}

In [7]:
svm_grid_parameters = [
  {'C': [0.01, 0.1, 1, 10, 100, 200, 400, 1000], 'kernel': ['linear']},
  {'C': [0.01, 0.1, 1, 10, 100, 200, 400, 1000], 'gamma': [100,10,1,1e-1,1e-2, 1e-3], 'kernel': ['rbf']},
 ]

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 1500, num = 2)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 80, num = 6)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [2, 4, 6]
bootstrap = [True, False]

rf_grid_parameters = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(rf_grid_parameters)

{'bootstrap': [True, False],
 'max_depth': [5, 20, 35, 50, 65, 80, None],
 'max_features': ['log2', 'sqrt'],
 'min_samples_leaf': [2, 4, 6],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [500, 1500]}


In [9]:
xgb_grid_parameters = {
    'learning_rate': [0.01,0.1],
    'max_depth': [3,6,10],
    'min_child_weight':[1,3,5], 
    'gamma':[0,1,5],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(5,11)], 
    'n_estimators': [500,1500]}

### Endpoint 1: Toxic

In [10]:
encoder_toxic = joblib.load('../encoder_models/encoder_toxic.joblib')

In [11]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'toxic', encoder = encoder_toxic)

H_t_knn = model_selection(knn_clf, knn_grid_parameters, a, c, scoring = 'roc_auc', cv=5, GridSearch = True)

Best parameters set found on development set: {'n_neighbors': 151, 'p': 1, 'weights': 'distance'}
Best score: 0.8786442988615353
Grid scores on development set:

0.839 (+/-0.050) for {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.838 (+/-0.052) for {'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
0.858 (+/-0.045) for {'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
0.856 (+/-0.050) for {'n_neighbors': 9, 'p': 2, 'weights': 'distance'}
0.868 (+/-0.048) for {'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
0.866 (+/-0.044) for {'n_neighbors': 15, 'p': 2, 'weights': 'distance'}
0.870 (+/-0.047) for {'n_neighbors': 19, 'p': 1, 'weights': 'distance'}
0.869 (+/-0.046) for {'n_neighbors': 19, 'p': 2, 'weights': 'distance'}
0.871 (+/-0.047) for {'n_neighbors': 25, 'p': 1, 'weights': 'distance'}
0.872 (+/-0.047) for {'n_neighbors': 25, 'p': 2, 'weights': 'distance'}
0.875 (+/-0.046) for {'n_neighbors': 35, 'p': 1, 'weights': 'distance'}
0.875 (+/-0.045) for {'n_neighbors': 35, 'p': 2, '

In [12]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'toxic', encoder = encoder_toxic)

H_t_svm = model_selection(svc, svm_grid_parameters, a, c, scoring = 'roc_auc', cv=5, GridSearch = True)

Best parameters set found on development set: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Best score: 0.8816975779192351
Grid scores on development set:

0.881 (+/-0.048) for {'C': 0.01, 'kernel': 'linear'}
0.881 (+/-0.049) for {'C': 0.1, 'kernel': 'linear'}
0.878 (+/-0.049) for {'C': 1, 'kernel': 'linear'}
0.877 (+/-0.049) for {'C': 10, 'kernel': 'linear'}
0.876 (+/-0.049) for {'C': 100, 'kernel': 'linear'}
0.876 (+/-0.049) for {'C': 200, 'kernel': 'linear'}
0.877 (+/-0.049) for {'C': 400, 'kernel': 'linear'}
0.876 (+/-0.049) for {'C': 1000, 'kernel': 'linear'}
0.509 (+/-0.006) for {'C': 0.01, 'gamma': 100, 'kernel': 'rbf'}
0.756 (+/-0.046) for {'C': 0.01, 'gamma': 10, 'kernel': 'rbf'}
0.825 (+/-0.044) for {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}
0.862 (+/-0.048) for {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}
0.863 (+/-0.051) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.851 (+/-0.054) for {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}
0.510 (+/-0.006) for {'C': 0.1, 'gamma': 100, 'k

In [13]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'toxic', encoder = encoder_toxic)

H_t_rf = model_selection(rf_clf, rf_grid_parameters, a, c, scoring = 'roc_auc', cv=5, GridSearch = False, n_iter=30,n_jobs=1)

Best parameters set found on development set: {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'log2', 'max_depth': 65, 'bootstrap': True}
Best score: 0.8814752606339683
Grid scores on development set:

0.881 (+/-0.048) for {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}
0.881 (+/-0.047) for {'n_estimators': 1500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 50, 'bootstrap': True}
0.881 (+/-0.049) for {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'log2', 'max_depth': 35, 'bootstrap': False}
0.881 (+/-0.048) for {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': False}
0.880 (+/-0.049) for {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 5, 'bootstrap': False}
0.881

In [14]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'toxic', encoder = encoder_toxic)

H_t_xgb = model_selection(xgb_clf, xgb_grid_parameters, a, c, scoring = 'roc_auc', cv=5, GridSearch = False, n_iter=30,n_jobs=1)

Best parameters set found on development set: {'subsample': 0.6, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 5, 'colsample_bytree': 0.6}
Best score: 0.8817171651619816
Grid scores on development set:

0.881 (+/-0.050) for {'subsample': 1.0, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 1, 'colsample_bytree': 0.9}
0.881 (+/-0.050) for {'subsample': 1.0, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.9}
0.871 (+/-0.047) for {'subsample': 1.0, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1.0}
0.881 (+/-0.048) for {'subsample': 0.7, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.01, 'gamma': 5, 'colsample_bytree': 0.5}
0.863 (+/-0.046) for {'subsample': 0.8, 'n_estimators': 1500, 'min_child_weight': 5, 'max_depth': 3, 'learning_r

### Endpoint 2: logLD50

In [15]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'logLD50_mmolkg', encoder = None)

H_ld50_rf = model_selection(rf_reg, rf_grid_parameters, a, c, scoring = 'neg_mean_squared_error', cv=5, GridSearch = False, n_iter=30,n_jobs=1)

Best parameters set found on development set: {'n_estimators': 1500, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': False}
Best score: -0.3066968307723924
Grid scores on development set:

-0.308 (+/-0.056) for {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 65, 'bootstrap': False}
-0.308 (+/-0.055) for {'n_estimators': 1500, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 35, 'bootstrap': True}
-0.308 (+/-0.055) for {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}
-0.309 (+/-0.055) for {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': 'log2', 'max_depth': 20, 'bootstrap': False}
-0.307 (+/-0.054) for {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 35, 'bootstrap': Fals

In [16]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'logLD50_mmolkg', encoder = None)

H_ld50_xgb = model_selection(xgb_reg, xgb_grid_parameters, a, c, scoring = 'neg_mean_squared_error', cv=5, GridSearch = False, n_iter=30,n_jobs=1)

Best parameters set found on development set: {'subsample': 0.6, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 1, 'colsample_bytree': 0.9}
Best score: -0.3072981768556339
Grid scores on development set:

-0.308 (+/-0.060) for {'subsample': 0.6, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.9}
-0.314 (+/-0.058) for {'subsample': 0.8, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 5, 'colsample_bytree': 0.9}
-0.315 (+/-0.059) for {'subsample': 1.0, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 5, 'colsample_bytree': 1.0}
-0.315 (+/-0.062) for {'subsample': 1.0, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 1, 'colsample_bytree': 0.5}
-0.318 (+/-0.066) for {'subsample': 0.7, 'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 3, 'le

In [17]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'logLD50_mmolkg', encoder = None)

H_ld50_svr = model_selection(svr, svm_grid_parameters, a, c, scoring = 'neg_mean_squared_error', cv=5, GridSearch = True)

Best parameters set found on development set: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Best score: -0.30772790955034346
Grid scores on development set:

-0.311 (+/-0.051) for {'C': 0.01, 'kernel': 'linear'}
-0.311 (+/-0.052) for {'C': 0.1, 'kernel': 'linear'}
-0.312 (+/-0.051) for {'C': 1, 'kernel': 'linear'}
-0.313 (+/-0.051) for {'C': 10, 'kernel': 'linear'}
-0.313 (+/-0.051) for {'C': 100, 'kernel': 'linear'}
-0.313 (+/-0.050) for {'C': 200, 'kernel': 'linear'}
-0.313 (+/-0.051) for {'C': 400, 'kernel': 'linear'}
-0.313 (+/-0.050) for {'C': 1000, 'kernel': 'linear'}
-0.844 (+/-0.335) for {'C': 0.01, 'gamma': 100, 'kernel': 'rbf'}
-0.844 (+/-0.335) for {'C': 0.01, 'gamma': 10, 'kernel': 'rbf'}
-0.770 (+/-0.318) for {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}
-0.384 (+/-0.111) for {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}
-0.345 (+/-0.066) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
-0.471 (+/-0.125) for {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}
-0.844 (+/-0.334) for {'C': 0.1, '

In [18]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'logLD50_mmolkg', encoder = None)

H_ld50_knn = model_selection(knn_reg, knn_grid_parameters, a, c, scoring = 'neg_mean_squared_error', cv=5, GridSearch = True)

Best parameters set found on development set: {'n_neighbors': 45, 'p': 2, 'weights': 'distance'}
Best score: -0.3181178447216808
Grid scores on development set:

-0.354 (+/-0.051) for {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
-0.352 (+/-0.049) for {'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
-0.333 (+/-0.049) for {'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
-0.335 (+/-0.046) for {'n_neighbors': 9, 'p': 2, 'weights': 'distance'}
-0.328 (+/-0.048) for {'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
-0.326 (+/-0.045) for {'n_neighbors': 15, 'p': 2, 'weights': 'distance'}
-0.324 (+/-0.052) for {'n_neighbors': 19, 'p': 1, 'weights': 'distance'}
-0.323 (+/-0.046) for {'n_neighbors': 19, 'p': 2, 'weights': 'distance'}
-0.323 (+/-0.047) for {'n_neighbors': 25, 'p': 1, 'weights': 'distance'}
-0.321 (+/-0.049) for {'n_neighbors': 25, 'p': 2, 'weights': 'distance'}
-0.322 (+/-0.048) for {'n_neighbors': 35, 'p': 1, 'weights': 'distance'}
-0.319 (+/-0.048) for {'n_neighbors': 3

### Endpoint 3: EPA Category

In [19]:
encoder_epa = joblib.load('../encoder_models/encoder_epa.joblib')

In [20]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'EPA_category', encoder = encoder_epa)

H_epa_rf = model_selection(rf_clf, rf_grid_parameters, a, c, scoring = 'f1_weighted', cv=5, GridSearch = False, n_iter=30,n_jobs=1)

Best parameters set found on development set: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 65, 'bootstrap': False}
Best score: 0.6389376458324004
Grid scores on development set:

0.627 (+/-0.073) for {'n_estimators': 1500, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': 'log2', 'max_depth': 5, 'bootstrap': False}
0.634 (+/-0.058) for {'n_estimators': 1500, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': None, 'bootstrap': True}
0.636 (+/-0.059) for {'n_estimators': 1500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': True}
0.635 (+/-0.058) for {'n_estimators': 1500, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 20, 'bootstrap': True}
0.637 (+/-0.050) for {'n_estimators': 1500, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}

In [21]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'EPA_category', encoder = encoder_epa)

H_epa_xgb = model_selection(xgb_clf, xgb_grid_parameters, a, c, scoring = 'f1_weighted', cv=5, GridSearch = False, n_iter=30,n_jobs=1)

Best parameters set found on development set: {'subsample': 0.9, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}
Best score: 0.6362382063966948
Grid scores on development set:

0.620 (+/-0.065) for {'subsample': 1.0, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1.0}
0.632 (+/-0.063) for {'subsample': 0.7, 'n_estimators': 1500, 'min_child_weight': 5, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 1, 'colsample_bytree': 1.0}
0.621 (+/-0.059) for {'subsample': 0.8, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.8}
0.636 (+/-0.059) for {'subsample': 0.9, 'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}
0.626 (+/-0.056) for {'subsample': 0.8, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 10, 'learni

In [22]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'EPA_category', encoder = encoder_epa)

H_epa_svc = model_selection(svc, svm_grid_parameters, a, c, scoring = 'f1_weighted', cv=5, GridSearch = True)

Best parameters set found on development set: {'C': 0.1, 'kernel': 'linear'}
Best score: 0.6338661563414092
Grid scores on development set:

0.632 (+/-0.067) for {'C': 0.01, 'kernel': 'linear'}
0.634 (+/-0.057) for {'C': 0.1, 'kernel': 'linear'}
0.633 (+/-0.050) for {'C': 1, 'kernel': 'linear'}
0.631 (+/-0.049) for {'C': 10, 'kernel': 'linear'}
0.631 (+/-0.048) for {'C': 100, 'kernel': 'linear'}
0.631 (+/-0.047) for {'C': 200, 'kernel': 'linear'}
0.631 (+/-0.046) for {'C': 400, 'kernel': 'linear'}
0.630 (+/-0.047) for {'C': 1000, 'kernel': 'linear'}
0.334 (+/-0.000) for {'C': 0.01, 'gamma': 100, 'kernel': 'rbf'}
0.334 (+/-0.000) for {'C': 0.01, 'gamma': 10, 'kernel': 'rbf'}
0.334 (+/-0.000) for {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}
0.547 (+/-0.069) for {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}
0.432 (+/-0.020) for {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}
0.334 (+/-0.000) for {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'}
0.334 (+/-0.000) for {'C': 0.1, 'gamma': 100, 'kernel': 'rbf

In [23]:
%%time
a,b,c,d,e = prepare_input(train_labels, train_Hfeatures, target = 'EPA_category', encoder = encoder_epa)

H_epa_knn = model_selection(knn_clf, knn_grid_parameters, a, c, scoring = 'f1_weighted', cv=5, GridSearch = True)

Best parameters set found on development set: {'n_neighbors': 55, 'p': 1, 'weights': 'distance'}
Best score: 0.6290994018747665
Grid scores on development set:

0.599 (+/-0.041) for {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.594 (+/-0.056) for {'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
0.612 (+/-0.049) for {'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
0.611 (+/-0.054) for {'n_neighbors': 9, 'p': 2, 'weights': 'distance'}
0.616 (+/-0.062) for {'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
0.618 (+/-0.053) for {'n_neighbors': 15, 'p': 2, 'weights': 'distance'}
0.619 (+/-0.057) for {'n_neighbors': 19, 'p': 1, 'weights': 'distance'}
0.621 (+/-0.061) for {'n_neighbors': 19, 'p': 2, 'weights': 'distance'}
0.624 (+/-0.059) for {'n_neighbors': 25, 'p': 1, 'weights': 'distance'}
0.626 (+/-0.066) for {'n_neighbors': 25, 'p': 2, 'weights': 'distance'}
0.627 (+/-0.061) for {'n_neighbors': 35, 'p': 1, 'weights': 'distance'}
0.626 (+/-0.063) for {'n_neighbors': 35, 'p': 2, 'w